In [1]:
import io
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

2021-10-10 22:03:19.782004: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import os  
import cv2
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# Loading np array from images
def make_dataset(labels, dir_path, IMG_SIZE = 150):
    # initial an empty list X to store image of np.array()
    X = []

    # initial an empty list Z to store labels/names of cat individauls
    Z = []
    for label in labels:
        DIR = dir_path + label
        for img in tqdm(os.listdir(DIR)):
            path = os.path.join(DIR,img)
            # reading images
            img = cv2.imread(path,cv2.IMREAD_COLOR)
            # resizing images to (150, 150, 3), 3 is the number of channels - RGB
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
            X.append(np.array(img))
            Z.append(str(label)) 
    ## Transform labels in Z to Y from class 0 to class 9, as 10 different cat individuals
    le=LabelEncoder()
    Y=le.fit_transform(Z)

    ## Transform and normalize X in the range of [0, 1]
    X=np.array(X)
    X=X/255.
    return X, Y

In [3]:
dir_path = '/kaggle/input/squirrel-monkey/Saimiri sciureus/'
IMG_SIZE = 150

# 44 monkey individuals, randomly select seen datset and unseen dataset
# make open set and unseen set 
def make_seen_unseen(num_classes, seen_ratio):
    arr = np.arange(num_classes)
    np.random.shuffle(arr)
    
    labels_seen = [str(i) for i in arr[:int(num_classes * seen_ratio)]]
    labels_unseen = [str(i) for i in arr[int(num_classes * seen_ratio):]]
    
    X_seen, Y_seen = make_dataset(labels_seen, dir_path)
    X_unseen, Y_unseen = make_dataset(labels_unseen, dir_path)
    Y_unseen = ['unseen'] * len(Y_unseen)
    Y_unseen = np.array(Y_unseen)
    return X_seen, Y_seen, X_unseen, Y_unseen

# Configuration settings

In [4]:
imsize = 150
EPOCHS = 20
batch_size = 256
embeddingDim = 128
num_individuals = 44

# Selecting seen and unseen

In [5]:
X_seen, Y_seen, X_unseen, Y_unseen = make_seen_unseen(44, 0.9)

100%|██████████| 7/7 [00:00<00:00, 47.02it/s]


# Train/test set split

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_seen, Y_seen, test_size = 0.2, random_state=2021)

X_val_unseen, X_test_unseen, Y_val_unseen, Y_test_unseen = train_test_split(X_unseen, Y_unseen, test_size = 0.5, random_state=2021)

# Stratified 5 fold CV

In [7]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=2021, shuffle=True)

# VGG16 Classification Model

In [8]:
def evaluate_vgg16(lr):
    base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                      input_shape = (imsize,imsize,3),
                      weights = 'imagenet')

    # freeze all the layers of VGG, so they won't be trained.
    for layer in base_model_16.layers:
        layer.trainable = False

    model_vgg_16 = tf.keras.models.Sequential([
        base_model_16,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1024, activation='relu'),
        tf.keras.layers.Dense(units=num_individuals, activation='softmax')
    ])

    model_vgg_16.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='sparse_categorical_crossentropy', metrics=['acc']) 

    model_vgg_16.fit(x=x_train, y=y_train,
                     epochs=EPOCHS,
                     batch_size=batch_size,
                     verbose=1)
    
    acc = model_vgg_16.evaluate(x_val, y_val)[1]
    
    return acc

In [9]:
lr = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

terms = {}
for i in lr:
    terms['{}'.format(i)] = []

for train_index, test_index in skf.split(X_train, Y_train):
    x_train,x_val,y_train,y_val = X_train[train_index], X_train[test_index], Y_train[train_index], Y_train[test_index]
    
    for i in lr:
        terms['{}'.format(i)].append(evaluate_vgg16(lr = i))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
2021-10-10 22:03:30.855354: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-10 22:03:30.858216: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-10 22:03:30.894941: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-10 22:03:30.895569: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88G

58892288/58889256 [==============================] - 2s 0us/step


2021-10-10 22:03:35.136932: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-10 22:03:35.147027: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000179999 Hz


Epoch 1/20


2021-10-10 22:03:35.890974: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-10-10 22:03:36.589328: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-10-10 22:03:36.619451: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


1/1 [==============================] - 12s 12s/step - loss: 3.9867 - acc: 0.0137
Epoch 2/20
1/1 [==============================] - 0s 159ms/step - loss: 3.9292 - acc: 0.0137
Epoch 3/20
1/1 [==============================] - 0s 157ms/step - loss: 3.8733 - acc: 0.0137
Epoch 4/20
1/1 [==============================] - 0s 159ms/step - loss: 3.8190 - acc: 0.0205
Epoch 5/20
1/1 [==============================] - 0s 143ms/step - loss: 3.7664 - acc: 0.0274
Epoch 6/20
1/1 [==============================] - 0s 147ms/step - loss: 3.7154 - acc: 0.0411
Epoch 7/20
1/1 [==============================] - 0s 147ms/step - loss: 3.6659 - acc: 0.0548
Epoch 8/20
1/1 [==============================] - 0s 159ms/step - loss: 3.6178 - acc: 0.0685
Epoch 9/20
1/1 [==============================] - 0s 146ms/step - loss: 3.5712 - acc: 0.0822
Epoch 10/20
1/1 [==============================] - 0s 147ms/step - loss: 3.5259 - acc: 0.1096
Epoch 11/20
1/1 [==============================] - 0s 151ms/step - loss: 3.4817 -

In [10]:
import pandas as pd
acc = pd.DataFrame(terms)
table_vgg16 = acc.agg(['mean', 'std']).T
table_vgg16

,mean,std
1e-05,0.174174,0.103147
0.0001,0.650000,0.033230
0.001,0.656156,0.084777
0.01,0.318018,0.121762
0.1,0.153003,0.049505


# Retrain VGG16 model on training set + val set, test it on test set

In [11]:
# choose best lr
lr = 0.001 

base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                      input_shape = (imsize,imsize,3),
                      weights = 'imagenet')

# freeze all the layers of VGG, so they won't be trained.
for layer in base_model_16.layers:
    layer.trainable = False

model_vgg_16 = tf.keras.models.Sequential([
    base_model_16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(units=num_individuals, activation='softmax')
])

model_vgg_16.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='sparse_categorical_crossentropy', metrics=['acc']) 

model_vgg_16.fit(x=X_train, y=Y_train,
                 epochs=EPOCHS,
                 batch_size=batch_size,
                 verbose=1)

Epoch 1/20
1/1 [==============================] - 7s 7s/step - loss: 4.2429 - acc: 0.0437
Epoch 2/20
1/1 [==============================] - 0s 178ms/step - loss: 4.4528 - acc: 0.0765
Epoch 3/20
1/1 [==============================] - 0s 177ms/step - loss: 4.2570 - acc: 0.2732
Epoch 4/20
1/1 [==============================] - 0s 179ms/step - loss: 3.9105 - acc: 0.4645
Epoch 5/20
1/1 [==============================] - 0s 179ms/step - loss: 3.4697 - acc: 0.5792
Epoch 6/20
1/1 [==============================] - 0s 179ms/step - loss: 2.9368 - acc: 0.6339
Epoch 7/20
1/1 [==============================] - 0s 178ms/step - loss: 2.4188 - acc: 0.7377
Epoch 8/20
1/1 [==============================] - 0s 171ms/step - loss: 1.8640 - acc: 0.7596
Epoch 9/20
1/1 [==============================] - 0s 174ms/step - loss: 1.3442 - acc: 0.8579
Epoch 10/20
1/1 [==============================] - 0s 180ms/step - loss: 0.9289 - acc: 0.8962
Epoch 11/20
1/1 [==============================] - 0s 191ms/step - loss:

In [12]:
acc_vgg16 = round(model_vgg_16.evaluate(X_test, Y_test)[1], 2)

print('Accuracy of VGG16 on the test set is {}'.format(acc_vgg16))

2/2 [==============================] - 1s 716ms/step - loss: 2.4723 - acc: 0.6304
Accuracy of VGG16 on the test set is 0.63


# Contrastive Loss

In [13]:
# Helper Function 

# Create positive pairs and negative pairs
import random
def create_pairs(images, labels):
    numClasses = 44
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive (0) or negative (1)
    np.random.seed(2021)
    pairImages = []
    pairLabels = []
    
    # calculate the total number of classes present in the dataset
    # and then build a list of indexes for each class label that
    # provides the indexes for all examples with a given label
    idx = [np.where(labels == i)[0] for i in range(44)]
    
    # loop voer all images
    for idxA in range(len(images)):
        # grab the current image and label belonging to the current iteration
        currentImage = images[idxA]
        label = labels[idxA]
        
        # randomly pick on an image that belongs to the *same* class label
        posId = random.choice(idx[label])
        posImage = images[posId]
        
        # prepare a positive pair and update the images and labels
        pairImages.append([currentImage, posImage])
        pairLabels.append([0])
        
        # grab the indices for each of the class labels *not* equal to
        # the current label and randomly pick an image corresponding
        # to a label *not* equal to the current label
        negId = np.where(labels != label)[0]
        negIdx = random.choice(negId)
        negImage = images[negIdx]
        
        # prepare a negative pair of images and update out lists
        pairImages.append([currentImage, negImage])
        pairLabels.append([1])
    
    return (np.array(pairImages), np.array(pairLabels))



# Function to calculate the distance between two images (Euclidean Distance used here)
import tensorflow.keras.backend as K
def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,
                       keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))


# contrastive loss function
def contrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean((1 - y) * squaredPreds + y * squaredMargin)
    # return the computed contrastive loss to the calling function
    return loss

## Closed Set

In [14]:
from sklearn.neighbors import KNeighborsClassifier

def evaluate_cl_closed_set(lr, k):
    base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                      input_shape = (imsize,imsize,3),
                                                      weights = 'imagenet')
    
    # freeze all the layers of VGG, so they won't be trained.
    for layer in base_model_16.layers:
        layer.trainable = False

    model_cl = tf.keras.models.Sequential([
        base_model_16,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1024, activation='relu'),
        tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
        tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
    ])

    
    imgA = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
    imgB = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
    
    featsA = model_cl(imgA)
    featsB = model_cl(imgB)
   
    distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])
    model = tf.keras.Model(inputs=[imgA, imgB], outputs=distance)
    model.compile(loss=contrastive_loss, optimizer=tf.keras.optimizers.Adam(lr))

    model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
              batch_size = batch_size,
              epochs=EPOCHS, 
              verbose=1)
    
    embedding_train_cl = []
    for i in range(len(y_train)):
        embedding_train_cl.append(model_cl(x_train[i].reshape(1, imsize, imsize, 3))[0])
    embedding_train_cl = np.array(embedding_train_cl, dtype=float) 

    knn_cl = KNeighborsClassifier(n_neighbors = k)
    knn_cl.fit(embedding_train_cl, y_train)
    
    x_test_embedding = model_cl(x_val)
    acc = round(knn_cl.score(x_test_embedding, y_val), 2)
    print('Accuracy on the val set with contrastive loss is {}'.format(acc))
    
    return acc

In [15]:
lr = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
k = [1, 3, 5]

terms = {}
for i in lr:
    for j in k:
        terms['{}_{}'.format(i, j)] = []

for train_index, test_index in skf.split(X_train, Y_train):
    x_train,x_val,y_train,y_val = X_train[train_index], X_train[test_index], Y_train[train_index], Y_train[test_index]
    
    (pairTrain, labelTrain) = create_pairs(x_train, y_train)
    
    for i in lr:
        for j in k:
            terms['{}_{}'.format(i, j)].append(evaluate_cl_closed_set(lr = i, k = j))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Epoch 1/20
2/2 [==============================] - 22s 3s/step - loss: 0.1950
Epoch 2/20
2/2 [==============================] - 1s 91ms/step - loss: 0.1860
Epoch 3/20
2/2 [==============================] - 1s 91ms/step - loss: 0.1771
Epoch 4/20
2/2 [==============================] - 1s 91ms/step - loss: 0.1688
Epoch 5/20
2/2 [==============================] - 1s 95ms/step - loss: 0.1637
Epoch 6/20
2/2 [==============================] - 1s 94ms/step - loss: 0.1576
Epoch 7/20
2/2 [==============================] - 1s 95ms/step - loss: 0.1498
Epoch 8/20
2/2 [==============================] - 1s 88ms/step - loss: 0.1465
Epoch 9/20
2/2 [==============================] - 1s 89ms/step - loss: 0.1455
Epoch 10/20
2/2 [==============================] - 1s 89ms/step - loss: 0.1406
Epoch 11/20
2/2 [==============================] - 1s 93ms/step - loss: 0.1386
Epoch 12/20
2/2 [==============================] - 1s 91ms/step - loss: 0.1363
Epoch 13/20
2/2 [==============================] - 1s 96ms/ste

In [16]:
import pandas as pd
acc = pd.DataFrame(terms)
table_cl_closed_set = acc.agg(['mean', 'std']).T
table_cl_closed_set

,mean,std
1e-05_1,0.650,0.037417
1e-05_3,0.584,0.063087
1e-05_5,0.562,0.047645
0.0001_1,0.704,0.062690
0.0001_3,0.634,0.073689
0.0001_5,0.622,0.072938
0.001_1,0.678,0.058052
0.001_3,0.572,0.044944
0.001_5,0.592,0.088431
0.01_1,0.542,0.057619


## Retrain Contrastive Loss on trainset + val set, and test it on test set

In [17]:
# choose best lr and k
lr = 0.0001
k = 1

base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                  input_shape = (imsize,imsize,3),
                                                  weights = 'imagenet')
    
# freeze all the layers of VGG, so they won't be trained.
for layer in base_model_16.layers:
    layer.trainable = False

model_cl = tf.keras.models.Sequential([
    base_model_16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
])

    
imgA = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
imgB = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
    
featsA = model_cl(imgA)
featsB = model_cl(imgB)
   
distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])
model = tf.keras.Model(inputs=[imgA, imgB], outputs=distance)
model.compile(loss=contrastive_loss, optimizer=tf.keras.optimizers.Adam(lr))

(pairTrain, labelTrain) = create_pairs(X_train, Y_train)

model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
          batch_size = batch_size,
          epochs=EPOCHS, 
          verbose=1)
    
embedding_train_cl = []
for i in range(len(Y_train)):
        embedding_train_cl.append(model_cl(X_train[i].reshape(1, imsize, imsize, 3))[0])
embedding_train_cl = np.array(embedding_train_cl, dtype=float) 

knn_cl = KNeighborsClassifier(n_neighbors = k)
knn_cl.fit(embedding_train_cl, Y_train)
    
x_test_embedding = model_cl(X_test)
acc_cl_closed_set = round(knn_cl.score(x_test_embedding, Y_test), 2)

print('Accuracy of Constractive Loss on test set is {}'.format(round(acc_cl_closed_set, 2)))

Epoch 1/20
2/2 [==============================] - 11s 9s/step - loss: 0.1951
Epoch 2/20
2/2 [==============================] - 1s 211ms/step - loss: 0.1409
Epoch 3/20
2/2 [==============================] - 1s 207ms/step - loss: 0.1230
Epoch 4/20
2/2 [==============================] - 1s 219ms/step - loss: 0.1065
Epoch 5/20
2/2 [==============================] - 1s 210ms/step - loss: 0.0879
Epoch 6/20
2/2 [==============================] - 1s 210ms/step - loss: 0.0824
Epoch 7/20
2/2 [==============================] - 1s 211ms/step - loss: 0.0649
Epoch 8/20
2/2 [==============================] - 1s 225ms/step - loss: 0.0586
Epoch 9/20
2/2 [==============================] - 1s 212ms/step - loss: 0.0532
Epoch 10/20
2/2 [==============================] - 1s 203ms/step - loss: 0.0447
Epoch 11/20
2/2 [==============================] - 1s 209ms/step - loss: 0.0404
Epoch 12/20
2/2 [==============================] - 1s 203ms/step - loss: 0.0379
Epoch 13/20
2/2 [==============================] - 

## Open Set

In [18]:
from sklearn.neighbors import KNeighborsClassifier

def evaluate_cl_open_set(lr=0.0001, k = 1, d_t = 0.5):
    base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                      input_shape = (imsize,imsize,3),
                                                      weights = 'imagenet')
    
    # freeze all the layers of VGG, so they won't be trained.
    for layer in base_model_16.layers:
        layer.trainable = False

    model_cl = tf.keras.models.Sequential([
        base_model_16,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1024, activation='relu'),
        tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
        tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
    ])

    
    imgA = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
    imgB = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
    
    featsA = model_cl(imgA)
    featsB = model_cl(imgB)
   
    distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])
    model = tf.keras.Model(inputs=[imgA, imgB], outputs=distance)
    model.compile(loss=contrastive_loss, optimizer=tf.keras.optimizers.Adam(lr))

    model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
              batch_size = batch_size,
              epochs=EPOCHS, 
              verbose=1)
    
    embedding_train_cl = []
    for i in range(len(y_train)):
        embedding_train_cl.append(model_cl(x_train[i].reshape(1, imsize, imsize, 3))[0])
    embedding_train_cl = np.array(embedding_train_cl, dtype=float) 

    knn_cl = KNeighborsClassifier(n_neighbors = k)
    knn_cl.fit(embedding_train_cl, y_train)
    
    #find the center point for each class in training set
    support_cl = []
    for i in np.unique(y_train):
        support_cl.append(np.mean(embedding_train_cl[y_train==i], axis=0))
        
    support_cl = np.array(support_cl, dtype=float)
    
    pred = []
    temp_x = np.append(x_val, X_val_unseen, axis=0)
    temp_y = np.append(y_val, Y_val_unseen, axis=0)
    
    arr = np.arange(temp_y.shape[0])
    np.random.shuffle(arr)
    
    temp_x = temp_x[arr]
    temp_y = temp_y[arr]
    
    for i in range(len(temp_y)):
        dists = []
        for j in range(len(np.unique(y_train))):
            embedding_test = model_cl(temp_x[i].reshape(1, 150, 150, 3))
            embedding_anchor = support_cl[j]
            dist = np.sum((embedding_test - embedding_anchor) ** 2) ** (1/2)
            dists.append(dist)
        if min(dists) >= d_t:
            pred.append('unseen')
        else:
            pred.append(knn_cl.predict(embedding_test)[0])

    pred = np.array(pred)
    
    acc_open = round(np.mean(pred == temp_y), 2)
    print('The accuracy on the val set with Contrastive Loss is {}'.format(acc_open))
    
    return acc_open

In [19]:
d_t = [0.4, 0.5, 0.6, 0.7, 0.8]

terms = {}
for i in d_t:
    terms['{}'.format(i)] = []

for train_index, test_index in skf.split(X_train, Y_train):
    x_train,x_val,y_train,y_val = X_train[train_index], X_train[test_index], Y_train[train_index], Y_train[test_index]
    
    (pairTrain, labelTrain) = create_pairs(x_train, y_train)
    
    for i in d_t:
        terms['{}'.format(i)].append(evaluate_cl_open_set(d_t = i))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Epoch 1/20
2/2 [==============================] - 2s 90ms/step - loss: 0.2003
Epoch 2/20
2/2 [==============================] - 1s 94ms/step - loss: 0.1566
Epoch 3/20
2/2 [==============================] - 1s 90ms/step - loss: 0.1247
Epoch 4/20
2/2 [==============================] - 1s 92ms/step - loss: 0.1187
Epoch 5/20
2/2 [==============================] - 1s 88ms/step - loss: 0.1015
Epoch 6/20
2/2 [==============================] - 1s 88ms/step - loss: 0.0833
Epoch 7/20
2/2 [==============================] - 1s 91ms/step - loss: 0.0762
Epoch 8/20
2/2 [==============================] - 1s 91ms/step - loss: 0.0669
Epoch 9/20
2/2 [==============================] - 1s 91ms/step - loss: 0.0604
Epoch 10/20
2/2 [==============================] - 1s 94ms/step - loss: 0.0538
Epoch 11/20
2/2 [==============================] - 1s 88ms/step - loss: 0.0492
Epoch 12/20
2/2 [==============================] - 1s 91ms/step - loss: 0.0447
Epoch 13/20
2/2 [==============================] - 1s 93ms/st

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


The accuracy on the val set with Contrastive Loss is 0.0
Epoch 1/20
2/2 [==============================] - 2s 92ms/step - loss: 0.2015
Epoch 2/20
2/2 [==============================] - 1s 92ms/step - loss: 0.1469
Epoch 3/20
2/2 [==============================] - 1s 91ms/step - loss: 0.1182
Epoch 4/20
2/2 [==============================] - 1s 93ms/step - loss: 0.1140
Epoch 5/20
2/2 [==============================] - 1s 95ms/step - loss: 0.1004
Epoch 6/20
2/2 [==============================] - 1s 90ms/step - loss: 0.0835
Epoch 7/20
2/2 [==============================] - 1s 90ms/step - loss: 0.0762
Epoch 8/20
2/2 [==============================] - 1s 91ms/step - loss: 0.0709
Epoch 9/20
2/2 [==============================] - 1s 94ms/step - loss: 0.0604
Epoch 10/20
2/2 [==============================] - 1s 95ms/step - loss: 0.0551
Epoch 11/20
2/2 [==============================] - 1s 92ms/step - loss: 0.0505
Epoch 12/20
2/2 [==============================] - 1s 93ms/step - loss: 0.0447
Epoc

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


The accuracy on the val set with Contrastive Loss is 0.0
Epoch 1/20
2/2 [==============================] - 2s 93ms/step - loss: 0.1841
Epoch 2/20
2/2 [==============================] - 1s 98ms/step - loss: 0.1493
Epoch 3/20
2/2 [==============================] - 1s 94ms/step - loss: 0.1110
Epoch 4/20
2/2 [==============================] - 1s 94ms/step - loss: 0.1035
Epoch 5/20
2/2 [==============================] - 1s 91ms/step - loss: 0.0852
Epoch 6/20
2/2 [==============================] - 1s 96ms/step - loss: 0.0741
Epoch 7/20
2/2 [==============================] - 1s 93ms/step - loss: 0.0678
Epoch 8/20
2/2 [==============================] - 1s 90ms/step - loss: 0.0601
Epoch 9/20
2/2 [==============================] - 1s 92ms/step - loss: 0.0529
Epoch 10/20
2/2 [==============================] - 1s 92ms/step - loss: 0.0502
Epoch 11/20
2/2 [==============================] - 1s 93ms/step - loss: 0.0459
Epoch 12/20
2/2 [==============================] - 1s 95ms/step - loss: 0.0403
Epoc

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


The accuracy on the val set with Contrastive Loss is 0.0
Epoch 1/20
2/2 [==============================] - 2s 93ms/step - loss: 0.1834
Epoch 2/20
2/2 [==============================] - 1s 94ms/step - loss: 0.1320
Epoch 3/20
2/2 [==============================] - 1s 100ms/step - loss: 0.1120
Epoch 4/20
2/2 [==============================] - 1s 94ms/step - loss: 0.0955
Epoch 5/20
2/2 [==============================] - 1s 92ms/step - loss: 0.0818
Epoch 6/20
2/2 [==============================] - 1s 96ms/step - loss: 0.0730
Epoch 7/20
2/2 [==============================] - 1s 96ms/step - loss: 0.0633
Epoch 8/20
2/2 [==============================] - 1s 93ms/step - loss: 0.0546
Epoch 9/20
2/2 [==============================] - 1s 92ms/step - loss: 0.0506
Epoch 10/20
2/2 [==============================] - 1s 90ms/step - loss: 0.0452
Epoch 11/20
2/2 [==============================] - 1s 92ms/step - loss: 0.0390
Epoch 12/20
2/2 [==============================] - 1s 92ms/step - loss: 0.0363
Epo

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [20]:
import pandas as pd
acc = pd.DataFrame(terms)
table_cl_open_set = acc.agg(['mean', 'std']).T
table_cl_open_set

,mean,std
0.4,0.578,0.041473
0.5,0.606,0.083546
0.6,0.110,0.245967
0.7,0.000,0.000000
0.8,0.000,0.000000


## Retrain Contrastive Loss on trainset + val set, and test it on test set

In [21]:
# choose the best value of d_t

lr=0.0001 
k = 1 
d_t = 0.5

base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                  input_shape = (imsize,imsize,3),
                                                  weights = 'imagenet')
    
# freeze all the layers of VGG, so they won't be trained.
for layer in base_model_16.layers:
    layer.trainable = False

model_cl = tf.keras.models.Sequential([
    base_model_16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
])

imgA = tf.keras.layers.Input(shape=((imsize, imsize, 3)))
imgB = tf.keras.layers.Input(shape=((imsize, imsize, 3)))

featsA = model_cl(imgA)
featsB = model_cl(imgB)

distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])

model = tf.keras.Model(inputs=[imgA, imgB], outputs=distance)

model.compile(loss=contrastive_loss, optimizer=tf.keras.optimizers.Adam(lr))

(pairTrain, labelTrain) = create_pairs(X_train, Y_train)

model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
        batch_size = batch_size,
        epochs=EPOCHS, 
        verbose=1)

embedding_train_cl = []
for i in range(len(Y_train)):
    embedding_train_cl.append(model_cl(X_train[i].reshape(1, imsize, imsize, 3))[0])

embedding_train_cl = np.array(embedding_train_cl, dtype=float) 

knn_cl = KNeighborsClassifier(n_neighbors = k)
knn_cl.fit(embedding_train_cl, Y_train)
    
#find the center point for each class in training set
support_cl = []
for i in np.unique(Y_train):
    support_cl.append(np.mean(embedding_train_cl[Y_train==i], axis=0))
        
support_cl = np.array(support_cl, dtype=float)
    
pred = []
temp_x = np.append(X_test, X_val_unseen, axis=0)
temp_y = np.append(Y_test, Y_val_unseen, axis=0)
    
arr = np.arange(temp_y.shape[0])
np.random.shuffle(arr)
    
temp_x = temp_x[arr]
temp_y = temp_y[arr]
    
for i in range(len(temp_y)):
    dists = []
    for j in range(len(np.unique(Y_train))):
        embedding_test = model_cl(temp_x[i].reshape(1, 150, 150, 3))
        embedding_anchor = support_cl[j]
        dist = np.sum((embedding_test - embedding_anchor) ** 2) ** (1/2)
        dists.append(dist)
    if min(dists) >= d_t:
            pred.append('unseen')
    else:
        pred.append(knn_cl.predict(embedding_test)[0])

pred = np.array(pred)
    
acc_cl_open_set = round(np.mean(pred == temp_y), 2)
print('The accuracy on the test set with Open Dataset of Contrastive Loss is {}'.format(acc_cl_open_set))

Epoch 1/20
2/2 [==============================] - 2s 214ms/step - loss: 0.1942
Epoch 2/20
2/2 [==============================] - 1s 206ms/step - loss: 0.1525
Epoch 3/20
2/2 [==============================] - 1s 204ms/step - loss: 0.1261
Epoch 4/20
2/2 [==============================] - 1s 217ms/step - loss: 0.1173
Epoch 5/20
2/2 [==============================] - 1s 220ms/step - loss: 0.0940
Epoch 6/20
2/2 [==============================] - 1s 238ms/step - loss: 0.0808
Epoch 7/20
2/2 [==============================] - 1s 233ms/step - loss: 0.0717
Epoch 8/20
2/2 [==============================] - 1s 219ms/step - loss: 0.0606
Epoch 9/20
2/2 [==============================] - 1s 213ms/step - loss: 0.0530
Epoch 10/20
2/2 [==============================] - 1s 205ms/step - loss: 0.0484
Epoch 11/20
2/2 [==============================] - 1s 211ms/step - loss: 0.0405
Epoch 12/20
2/2 [==============================] - 1s 210ms/step - loss: 0.0355
Epoch 13/20
2/2 [==============================] 

# Triplet Loss

## Closed Set

In [22]:
def evaluate_tl_closed_set(lr, k):
    base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                      input_shape = (imsize, imsize, 3),
                                                      weights = 'imagenet')

    # freeze all the layers of VGG, so they won't be trained.
    for layer in base_model_16.layers:
        layer.trainable = False

    model_tl = tf.keras.models.Sequential([
        base_model_16,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1024, activation='relu'),
        tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
        tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
    ])

    model_tl.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss=tfa.losses.TripletSemiHardLoss())

    model_tl.fit(x=x_train, y= y_train,
                 batch_size=batch_size,
                 epochs=EPOCHS,
                 verbose=1) 
        
    embedding_train_tl = []
    for i in range(len(y_train)):
        embedding_train_tl.append(model_tl(x_train[i].reshape(1, imsize, imsize, 3))[0])
        
    embedding_train_tl = np.array(embedding_train_tl, dtype=float) 

    knn_tl = KNeighborsClassifier(n_neighbors = k)
    knn_tl.fit(embedding_train_tl, y_train)
    
    x_test_embedding = model_tl(x_val)
    acc = round(knn_tl.score(x_test_embedding, y_val), 2)
    print('Accuracy on the the val set with Tripolet Loss is {}'.format(acc))
    
    return acc

In [23]:
lr = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
k = [1, 3, 5]

terms = {}
for i in lr:
    for j in k:
        terms['{}_{}'.format(i, j)] = []
        
for train_index, test_index in skf.split(X_train, Y_train):
    x_train,x_val,y_train,y_val = X_train[train_index], X_train[test_index], Y_train[train_index], Y_train[test_index]
    
    for i in lr:
        for j in k:
            terms['{}_{}'.format(i, j)].append(evaluate_tl_closed_set(lr = i, k = j))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.9785
Epoch 2/20
1/1 [==============================] - 0s 145ms/step - loss: 0.9770
Epoch 3/20
1/1 [==============================] - 0s 149ms/step - loss: 0.9750
Epoch 4/20
1/1 [==============================] - 0s 152ms/step - loss: 0.9731
Epoch 5/20
1/1 [==============================] - 0s 145ms/step - loss: 0.9710
Epoch 6/20
1/1 [==============================] - 0s 144ms/step - loss: 0.9686
Epoch 7/20
1/1 [==============================] - 0s 149ms/step - loss: 0.9662
Epoch 8/20
1/1 [==============================] - 0s 183ms/step - loss: 0.9639
Epoch 9/20
1/1 [==============================] - 0s 164ms/step - loss: 0.9611
Epoch 10/20
1/1 [==============================] - 0s 171ms/step - loss: 0.9579
Epoch 11/20
1/1 [==============================] - 0s 152ms/step - loss: 0.9549
Epoch 12/20
1/1 [==============================] - 0s 148ms/step - loss: 0.9514
Epoch 13/20
1/1 [==============================] - 0

In [24]:
import pandas as pd
acc = pd.DataFrame(terms)
table_tl_closed_set = acc.agg(['mean', 'std']).T
table_tl_closed_set

,mean,std
1e-05_1,0.676,0.076026
1e-05_3,0.606,0.053198
1e-05_5,0.606,0.033615
0.0001_1,0.732,0.036332
0.0001_3,0.678,0.021679
0.0001_5,0.658,0.016432
0.001_1,0.748,0.081670
0.001_3,0.706,0.057271
0.001_5,0.712,0.050695
0.01_1,0.710,0.076485


## Retrain Triplet Loss on trainset + val set, and test it on test set

In [25]:
# choose best lr and k
lr = 0.001
k = 1

base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                  input_shape = (imsize, imsize, 3),
                                                  weights = 'imagenet')

# freeze all the layers of VGG, so they won't be trained.
for layer in base_model_16.layers:
        layer.trainable = False

model_tl = tf.keras.models.Sequential([
    base_model_16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
])

model_tl.compile(
    optimizer=tf.keras.optimizers.Adam(lr),
    loss=tfa.losses.TripletSemiHardLoss())

model_tl.fit(x=X_train, y= Y_train,
             batch_size=batch_size,
             epochs=EPOCHS,
             verbose=1) 
        
embedding_train_tl = []
for i in range(len(Y_train)):
    embedding_train_tl.append(model_tl(X_train[i].reshape(1, imsize, imsize, 3))[0])
        
embedding_train_tl = np.array(embedding_train_tl, dtype=float) 

knn_tl = KNeighborsClassifier(n_neighbors = k)
knn_tl.fit(embedding_train_tl, Y_train)
    
x_test_embedding = model_tl(X_test)
acc_tl_closed_set = round(knn_tl.score(x_test_embedding, Y_test), 2)

print('Accuracy of Triplet Loss on the test set is {}'.format(round(acc_tl_closed_set, 2)))

Epoch 1/20
1/1 [==============================] - 1s 777ms/step - loss: 0.9777
Epoch 2/20
1/1 [==============================] - 0s 178ms/step - loss: 0.9042
Epoch 3/20
1/1 [==============================] - 0s 197ms/step - loss: 0.7763
Epoch 4/20
1/1 [==============================] - 0s 175ms/step - loss: 0.7252
Epoch 5/20
1/1 [==============================] - 0s 185ms/step - loss: 0.5549
Epoch 6/20
1/1 [==============================] - 0s 194ms/step - loss: 0.4972
Epoch 7/20
1/1 [==============================] - 0s 194ms/step - loss: 0.4168
Epoch 8/20
1/1 [==============================] - 0s 174ms/step - loss: 0.3199
Epoch 9/20
1/1 [==============================] - 0s 178ms/step - loss: 0.2519
Epoch 10/20
1/1 [==============================] - 0s 180ms/step - loss: 0.2060
Epoch 11/20
1/1 [==============================] - 0s 175ms/step - loss: 0.1605
Epoch 12/20
1/1 [==============================] - 0s 180ms/step - loss: 0.1427
Epoch 13/20
1/1 [==============================] 

## Open set

In [26]:
def evaluate_tl_open_set(lr = 0.001, k = 1, d_t =0.5):
    base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                      input_shape = (imsize, imsize, 3),
                                                      weights = 'imagenet')

    # freeze all the layers of VGG, so they won't be trained.
    for layer in base_model_16.layers:
        layer.trainable = False

    model_tl = tf.keras.models.Sequential([
        base_model_16,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1024, activation='relu'),
        tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
        tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
    ])

    model_tl.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss=tfa.losses.TripletSemiHardLoss())

    model_tl.fit(x=x_train, y= y_train,
                 batch_size=batch_size,
                 epochs=EPOCHS,
                 verbose=1) 
        
    embedding_train_tl = []
    for i in range(len(y_train)):
        embedding_train_tl.append(model_tl(x_train[i].reshape(1, imsize, imsize, 3))[0])
        
    embedding_train_tl = np.array(embedding_train_tl, dtype=float) 

    knn_tl = KNeighborsClassifier(n_neighbors = k)
    knn_tl.fit(embedding_train_tl, y_train)
    
    #find the center point for each class in training set
    support_tl = []
    for i in np.unique(y_train):
        support_tl.append(np.mean(embedding_train_tl[y_train==i], axis=0))
        
    support_tl = np.array(support_tl, dtype=float)
    
    pred = []
    temp_x = np.append(x_val, X_unseen, axis=0)
    temp_y = np.append(y_val, Y_unseen, axis=0)
    
    arr = np.arange(temp_y.shape[0])
    np.random.shuffle(arr)
    
    temp_x = temp_x[arr]
    temp_y = temp_y[arr]
    
    for i in range(len(temp_y)):
        dists = []
        for j in range(len(np.unique(y_train))):
            embedding_test = model_tl(temp_x[i].reshape(1, 150, 150, 3))
            embedding_anchor = support_tl[j]
            dist = np.sum((embedding_test - embedding_anchor) ** 2) ** (1/2)
            dists.append(dist)
        if min(dists) >= d_t:
            pred.append('unseen')
        else:
            pred.append(knn_tl.predict(embedding_test)[0])

    pred = np.array(pred)
    
    acc_open = round(np.mean(pred == temp_y), 2)
    print('The accuracy on the Open Dataset with triplet loss is {}'.format(acc_open))
    
    return acc_open

In [27]:
d_t = [0.4, 0.5, 0.6, 0.7, 0.8]


terms = {}
for i in d_t:
    terms['{}'.format(i)] = []

for train_index, test_index in skf.split(X_train, Y_train):
    x_train,x_val,y_train,y_val = X_train[train_index], X_train[test_index], Y_train[train_index], Y_train[test_index]
    
    for i in d_t:
        terms['{}'.format(i)].append(evaluate_tl_open_set(d_t = i))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Epoch 1/20
1/1 [==============================] - 1s 1s/step - loss: 0.9824
Epoch 2/20
1/1 [==============================] - 0s 152ms/step - loss: 0.8884
Epoch 3/20
1/1 [==============================] - 0s 149ms/step - loss: 0.7425
Epoch 4/20
1/1 [==============================] - 0s 157ms/step - loss: 0.6630
Epoch 5/20
1/1 [==============================] - 0s 142ms/step - loss: 0.4961
Epoch 6/20
1/1 [==============================] - 0s 158ms/step - loss: 0.4054
Epoch 7/20
1/1 [==============================] - 0s 150ms/step - loss: 0.3858
Epoch 8/20
1/1 [==============================] - 0s 148ms/step - loss: 0.2492
Epoch 9/20
1/1 [==============================] - 0s 160ms/step - loss: 0.2009
Epoch 10/20
1/1 [==============================] - 0s 150ms/step - loss: 0.1994
Epoch 11/20
1/1 [==============================] - 0s 144ms/step - loss: 0.1340
Epoch 12/20
1/1 [==============================] - 0s 143ms/step - loss: 0.1068
Epoch 13/20
1/1 [==============================] - 0

In [28]:
import pandas as pd
acc = pd.DataFrame(terms)
table_tl_open_set = acc.agg(['mean', 'std']).T
table_tl_open_set

,mean,std
0.4,0.598,0.042071
0.5,0.658,0.025884
0.6,0.710,0.043589
0.7,0.724,0.036469
0.8,0.654,0.043932


## Retrain Triplet Loss on trainset + val set, and test it on test set

In [29]:
lr = 0.001
k = 1
d_t = 0.8

base_model_16 = tf.keras.applications.vgg16.VGG16(include_top=False,
                                                  input_shape = (imsize, imsize, 3),
                                                  weights = 'imagenet')

# freeze all the layers of VGG, so they won't be trained.
for layer in base_model_16.layers:
    layer.trainable = False

model_tl = tf.keras.models.Sequential([
    base_model_16,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.Dense(embeddingDim, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
])

model_tl.compile(
    optimizer=tf.keras.optimizers.Adam(lr),
    loss=tfa.losses.TripletSemiHardLoss())

model_tl.fit(x = X_train, y = Y_train,
             batch_size=batch_size,
             epochs=EPOCHS,
             verbose=1) 
        
embedding_train_tl = []
for i in range(len(Y_train)):
    embedding_train_tl.append(model_tl(X_train[i].reshape(1, imsize, imsize, 3))[0])
        
embedding_train_tl = np.array(embedding_train_tl, dtype=float) 

knn_tl = KNeighborsClassifier(n_neighbors = k)
knn_tl.fit(embedding_train_tl, Y_train)
    
#find the center point for each class in training set
support_tl = []
for i in np.unique(Y_train):
    support_tl.append(np.mean(embedding_train_tl[Y_train==i], axis=0))
        
support_tl = np.array(support_tl, dtype=float)
    
pred = []
temp_x = np.append(X_test, X_unseen, axis=0)
temp_y = np.append(Y_test, Y_unseen, axis=0)
    
arr = np.arange(temp_y.shape[0])
np.random.shuffle(arr)
    
temp_x = temp_x[arr]
temp_y = temp_y[arr]
    
for i in range(len(temp_y)):
    dists = []
    for j in range(len(np.unique(Y_train))):
        embedding_test = model_tl(temp_x[i].reshape(1, 150, 150, 3))
        embedding_anchor = support_tl[j]
        dist = np.sum((embedding_test - embedding_anchor) ** 2) ** (1/2)
        dists.append(dist)
    if min(dists) >= d_t:
        pred.append('unseen')
    else:
        pred.append(knn_tl.predict(embedding_test)[0])

pred = np.array(pred)
    
acc_tl_open_set = round(np.mean(pred == temp_y), 2)
print('The accuracy on the test set with Open Dataset of Triplet Loss is {}'.format(acc_tl_open_set))

Epoch 1/20
1/1 [==============================] - 1s 775ms/step - loss: 0.9826
Epoch 2/20
1/1 [==============================] - 0s 189ms/step - loss: 0.9095
Epoch 3/20
1/1 [==============================] - 0s 177ms/step - loss: 0.7423
Epoch 4/20
1/1 [==============================] - 0s 186ms/step - loss: 0.7025
Epoch 5/20
1/1 [==============================] - 0s 186ms/step - loss: 0.5618
Epoch 6/20
1/1 [==============================] - 0s 188ms/step - loss: 0.4633
Epoch 7/20
1/1 [==============================] - 0s 176ms/step - loss: 0.3956
Epoch 8/20
1/1 [==============================] - 0s 191ms/step - loss: 0.2980
Epoch 9/20
1/1 [==============================] - 0s 196ms/step - loss: 0.2486
Epoch 10/20
1/1 [==============================] - 0s 175ms/step - loss: 0.1901
Epoch 11/20
1/1 [==============================] - 0s 194ms/step - loss: 0.1541
Epoch 12/20
1/1 [==============================] - 0s 194ms/step - loss: 0.1346
Epoch 13/20
1/1 [==============================] 

# Summary on Closed Set

In [30]:
# After tuning hps (lr and k), the best accuracy on val set.
best_vgg16 = table_vgg16.loc['0.0001']
best_cl = table_cl_closed_set.loc['0.0001_1']
best_tl = table_tl_closed_set.loc['0.001_1']

pd.DataFrame({'VGG16_(LR = 0.0001)': best_vgg16,
              'Contrastive_Loss(LR = 0.0001, k = 1)': best_cl, 
              'Triplet_Loss(LR = 0.001, k = 1)': best_tl
             })

,VGG16_(LR = 0.0001),"Contrastive_Loss(LR = 0.0001, k = 1)","Triplet_Loss(LR = 0.001, k = 1)"
mean,0.65000,0.70400,0.74800
std,0.03323,0.06269,0.08167


In [31]:
# Accuracy on test set

pd.DataFrame([acc_vgg16, acc_cl_closed_set, acc_tl_closed_set], index = ['VGG16', 'Contrastive_Loss', 'Triplet_Loss'], columns = ['Accuracy']).T

,VGG16,Contrastive_Loss,Triplet_Loss
Accuracy,0.63,0.72,0.72


# Summary on Open Set

In [32]:
# After tuning hps (d_t), the best accuracy on val set.

best_cl = table_cl_open_set.loc['0.5']
best_tl = table_tl_open_set.loc['0.8']

pd.DataFrame({'Contrastive_Loss(d_t = 0.5)': best_cl, 
              'Triplet_Loss(d_t = 0.8)': best_tl
             })

,Contrastive_Loss(d_t = 0.5),Triplet_Loss(d_t = 0.8)
mean,0.606000,0.654000
std,0.083546,0.043932


In [33]:
# Accuracy on test set

pd.DataFrame([acc_cl_open_set, acc_tl_open_set], index = ['Contrastive_Loss', 'Triplet_Loss'], columns = ['Accuracy']).T

,Contrastive_Loss,Triplet_Loss
Accuracy,0.6,0.68
